## 1) Installing required libraries

In this cell I'm just preparing the environment for the whole extraction pipeline.  
I upgrade `pip` and install the libraries I need to read PowerPoint, Word, and Excel files, work with tables using pandas, and call the OpenAI model.  
Nothing complicated here — just making sure everything is ready for the next steps.

In [0]:
%pip install --upgrade pip
%pip install openai==1.55.3 httpx==0.27.2 python-pptx python-docx openpyxl pandas

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

### 2) Initializing the Azure OpenAI client

This cell sets up the Azure OpenAI configuration by defining the endpoint, API key, and client object used for sending requests to the GPT model.  
These settings allow the notebook to communicate with the Azure OpenAI deployment throughout the extraction and analysis workflow.  
For production use, the API key should be stored securely in a Databricks secret scope rather than directly in the notebook.


In [0]:
from openai import AzureOpenAI

# Read secrets from Databricks Secret Scope
AZURE_OPENAI_ENDPOINT = dbutils.secrets.get("itera-secrets", "AZURE-OPENAI-ENDPOINT")
AZURE_OPENAI_KEY = dbutils.secrets.get("itera-secrets", "AZURE-OPENAI-KEY")

client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    api_version="2025-01-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

### Configuring SAS access for Azure Data Lake (ABFS)

This cell sets up Spark to authenticate against Azure Data Lake Gen2 using a SAS token.  
The configuration enables the notebook to read and process files stored in the specified container by registering the SAS token with the ABFS driver.  
A base `abfss://` path is also built here, which is reused in the following cells when loading documents from storage.  
For long-term or production use, the SAS token should be stored securely (for example, in a Databricks secret scope) instead of being written directly in the notebook.


In [0]:
# Read SAS token securely from Databricks secret scope
SAS_TOKEN = dbutils.secrets.get("itera-secrets", "ADLS-SAS-TOKEN")
ACCOUNT   = "iterastorerm"
CONTAINER = "itera-extraction-project"

# Configure Spark to use the SAS for ABFS (Azure Data Lake Gen2)
spark.conf.set(f"fs.azure.account.auth.type.{ACCOUNT}.dfs.core.windows.net", "SAS")
spark.conf.set(
    f"fs.azure.sas.token.provider.type.{ACCOUNT}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider"
)
# lstrip('?') removes the leading '?' so Spark gets only the token body
spark.conf.set(
    f"fs.azure.sas.fixed.token.{ACCOUNT}.dfs.core.windows.net",
    SAS_TOKEN.lstrip("?")
)

# Build the base ABFSS path used in the next cells
BASE_ABFSS = f"abfss://{CONTAINER}@{ACCOUNT}.dfs.core.windows.net"
INPUT_DIR  = BASE_ABFSS
print("Configured. Base path:", BASE_ABFSS)


In [0]:
# Listing files in the storage container
display(dbutils.fs.ls(INPUT_DIR))

path,name,size,modificationTime
abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/TG3 Project Plan + Establishment v1.2.pptx,TG3 Project Plan + Establishment v1.2.pptx,535197,1764943162000
abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/all_content.txt,all_content.txt,23812,1764945519000
abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/output.json,output.json,1415,1764945524000
abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/practice/,practice/,0,0
abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/resources_kpi.json,resources_kpi.json,20,1763563201000



### Defining helper functions for reading DOCX, XLSX, and PPTX files

In the next few cells, helper functions are defined for extracting content from Word, Excel, and PowerPoint files stored in Azure Data Lake.

These functions together make it possible to:
- Load binary files directly from ABFS using Spark.
- Read text and tables from Word (.docx) files.
- Read data from all sheets inside Excel (.xlsx) files.
- Parse PowerPoint (.pptx) slides and extract slide text, native tables, slide notes, and detect embedded Excel objects.
- When embedded Excel workbooks exist inside a slide, extract them and place the content directly under the slide where they appear, so the structure of the presentation is kept.

Even though the functions are split across separate cells, they work together as one toolbox and form the core of the data-extraction pipeline, allowing the notebook to turn different document formats into structured, readable output.


In [0]:
# Importing all required libraries for reading, parsing, and processing DOCX, XLSX, and PPTX files.
import io
import re
import pandas as pd
import os
import xml.etree.ElementTree as ET
import zipfile
from docx import Document
from openpyxl import load_workbook
from pptx import Presentation

In [0]:
# AZURE DATA LAKE FILE READER
def _read_bytes_abfs(path):
    """
    Read a file from Azure Blob File System (ABFS) and return its raw bytes.
    
    Args:
        path (str): ABFS path to the file
        
    Returns:
        bytes: Raw file content as bytes
        
    Raises:
        FileNotFoundError: If the file doesn't exist at the given path
    """
    df = spark.read.format("binaryFile").load(path)
    rows = df.collect()
    
    if not rows:
        raise FileNotFoundError(f"File not found: {path}")
    
    return rows[0]["content"]

In [0]:
# WORD DOCUMENT (.docx) EXTRACTION
def read_docx_text(abfs_path, include_tables=False):
    """
    Extract all text content from a Word document, optionally including tables.
    
    Args:
        abfs_path (str): Path to the .docx file in Azure Data Lake
        include_tables (bool): If True, extracts table content as well
        
    Returns:
        str: Extracted text with paragraphs and tables (if requested)
        
    Example:
        >>> text = read_docx_text("abfs://container/document.docx", include_tables=True)
        >>> print(text)
    """
    # Load the Word document from Azure
    data = _read_bytes_abfs(abfs_path)
    doc = Document(io.BytesIO(data))
    
    content = []
    
    # Extract all paragraphs
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        if text:  # Only include non-empty paragraphs
            content.append(text)
    
    # Extract tables if requested
    if include_tables:
        for i, table in enumerate(doc.tables):
            content.append(f"\n--- Table {i + 1} ---")
            
            for row in table.rows:
                # Get text from each cell in the row
                cells = [cell.text.strip() for cell in row.cells]
                
                # Only include rows that have some content
                if any(cells):
                    content.append(" | ".join(cells))
    
    return "\n".join(content)

In [0]:
# EXCEL WORKBOOK (.xlsx) EXTRACTION
def read_xlsx_sheets(abfs_path):
    """
    Read all sheets from an Excel workbook into pandas DataFrames.
    
    Args:
        abfs_path (str): Path to the .xlsx file in Azure Data Lake
        
    Returns:
        dict: Dictionary with sheet names as keys and DataFrames as values
              Example: {'Sheet1': DataFrame, 'Sales Data': DataFrame}
              
    Example:
        >>> sheets = read_xlsx_sheets("abfs://container/data.xlsx")
        >>> for sheet_name, df in sheets.items():
        ...     print(f"{sheet_name}: {len(df)} rows")
    """
    # Load the Excel file from Azure
    data = _read_bytes_abfs(abfs_path)
    workbook = load_workbook(io.BytesIO(data), data_only=True)

    sheets = {}
    
    for sheet in workbook.worksheets:
        # Get all cell values from the sheet
        rows = list(sheet.values)
        
        if not rows:
            continue  # Skip empty sheets
        
        # First row is assumed to be the header
        header = rows[0]
        data_rows = rows[1:]
        
        # Handle cases where headers are missing or all None
        if not header or all(h is None for h in header):
            num_cols = len(data_rows[0]) if data_rows else 0
            header = [f"Column_{i+1}" for i in range(num_cols)]
        
        # Create DataFrame from the data
        df = pd.DataFrame(data_rows, columns=header)
        
        # Remove completely empty rows (all NaN values)
        df = df.dropna(how='all')
        
        sheets[sheet.title] = df
    
    return sheets

In [0]:
# POWERPOINT EMBEDDED EXCEL EXTRACTION (STANDALONE)
def read_pptx_embedded_excels(abfs_path):
    """
    Extract all embedded Excel workbooks from a PowerPoint file.
    
    This function is useful when you only want the Excel data without
    the PowerPoint text content.
    
    Args:
        abfs_path (str): Path to the .pptx file in Azure Data Lake
        
    Returns:
        dict: Nested dictionary structure:
              {
                  'ppt/embeddings/embeddedWorkbook1.xlsx': {
                      'Sheet1': DataFrame,
                      'Sheet2': DataFrame
                  },
                  'ppt/embeddings/embeddedWorkbook2.xlsx': {
                      'Data': DataFrame
                  }
              }
              
    Example:
        >>> embedded = read_pptx_embedded_excels("abfs://container/presentation.pptx")
        >>> for file_path, sheets in embedded.items():
        ...     print(f"Found: {file_path}")
    """
    # Load the PowerPoint file from Azure
    data = _read_bytes_abfs(abfs_path)
    embedded = {}

    # PowerPoint files are actually ZIP archives - open as such
    with zipfile.ZipFile(io.BytesIO(data)) as z:
        # Find all embedded Excel files in the ZIP structure
        excel_files = [
            name for name in z.namelist()
            if name.startswith("ppt/embeddings/")
            and (name.endswith(".xlsx") or name.endswith(".xlsm"))
        ]

        # Process each embedded Excel file
        for name in excel_files:
            wb_bytes = z.read(name)
            wb = load_workbook(io.BytesIO(wb_bytes), data_only=True)

            sheet_dfs = {}
            
            for sheet in wb.worksheets:
                rows = list(sheet.values)
                
                if not rows:
                    continue  # Skip empty sheets
                
                header = rows[0]
                data_rows = rows[1:]

                # Generate generic headers if missing
                if not header or all(h is None for h in header):
                    num_cols = len(data_rows[0]) if data_rows else 0
                    header = [f"Column_{i+1}" for i in range(num_cols)]

                df = pd.DataFrame(data_rows, columns=header)
                df = df.dropna(how="all")
                sheet_dfs[sheet.title] = df

            # Only include workbooks that have at least one non-empty sheet
            if sheet_dfs:
                embedded[name] = sheet_dfs

    return embedded

In [0]:
def extract_smartart_texts(slide):
    """
    Extract raw SmartArt texts (Sponsor, Project Manager, names, etc.)
    from the SmartArt XML parts of a slide.
    Returns a flat list of strings in the same order as in the SmartArt pane.
    """
    smartart_texts = []
    slide_part = slide.part

    for rel in slide_part.rels.values():
        if "diagram" in rel.reltype:  # SmartArt / diagram parts
            try:
                part = rel.target_part
                root = ET.fromstring(part.blob)
                for el in root.iter():
                    # SmartArt text nodes are typically <dgm:t>
                    if el.tag.endswith("}t") and el.text and el.text.strip():
                        smartart_texts.append(el.text.strip())
            except Exception:
                continue

    return smartart_texts

In [0]:
def read_pptx_text(abfs_path, include_embedded_excels=True, max_rows=10):
    """
    Read text from a PowerPoint presentation.
    - Extracts slide text (including grouped shapes), native tables, notes.
    - Extracts embedded Excel workbooks and attaches them under the slide.
    - Extracts SmartArt / diagram text per slide and outputs JSON-structured blocks.
    """
    data = _read_bytes_abfs(abfs_path)

    # --------- Step 1: Load all embedded Excel workbooks into a cache ---------
    embedded_cache = {}
    if include_embedded_excels:
        with zipfile.ZipFile(io.BytesIO(data)) as z:
            excel_files = [
                name for name in z.namelist()
                if name.startswith("ppt/embeddings/")
                and (name.endswith(".xlsx") or name.endswith(".xlsm"))
            ]
            
            for name in excel_files:
                wb_bytes = z.read(name)
                wb = load_workbook(io.BytesIO(wb_bytes), data_only=True)

                sheet_dfs = {}
                for sheet in wb.worksheets:
                    rows = list(sheet.values)
                    if not rows:
                        continue

                    header = rows[0]
                    data_rows = rows[1:]

                    if not header or all(h is None for h in header):
                        num_cols = len(data_rows[0]) if data_rows else 0
                        header = [f"Column_{i+1}" for i in range(num_cols)]

                    df = pd.DataFrame(data_rows, columns=header)
                    df = df.dropna(how="all")
                    sheet_dfs[sheet.title] = df

                if sheet_dfs:
                    # Store by filename for lookup
                    embedded_cache[name] = sheet_dfs

    # --------- Step 2: Parse slides and match embedded files via relationships ---------
    prs = Presentation(io.BytesIO(data))
    content = []

    for slide_num, slide in enumerate(prs.slides, start=1):
        content.append(f"\n--- Slide {slide_num} ---")
        slide_has_content = False

        # Helper to extract embedded file path from shape relationships
        def get_embedded_excel_path(shape, slide):
            """Get the path to embedded Excel file for this specific shape"""
            try:
                # Check if shape has an OLE object
                if "oleObj" not in shape._element.xml:
                    return None
                
                # Parse the shape XML to find the relationship ID
                shape_xml = shape._element.xml
                
                # Look for r:id or r:embed attributes
                rid_match = re.search(r'r:id="(rId\d+)"', shape_xml)
                if not rid_match:
                    rid_match = re.search(r'r:embed="(rId\d+)"', shape_xml)
                
                if not rid_match:
                    return None
                
                rid = rid_match.group(1)
                
                # Get the actual file path from slide relationships
                slide_part = slide.part
                if rid in slide_part.rels:
                    rel = slide_part.rels[rid]
                    target = rel.target_ref
                    
                    # Convert relative path to full path
                    if target.startswith("../embeddings/"):
                        return f"ppt/embeddings/{target.split('/')[-1]}"
                    elif target.startswith("embeddings/"):
                        return f"ppt/{target}"
                    
            except Exception as e:
                pass
            
            return None

        # Helper to handle normal shapes + grouped shapes
        def extract_from_shape(shape):
            nonlocal slide_has_content

            # If this is a group shape, recurse into its children
            if hasattr(shape, "shapes") and len(shape.shapes) > 0:
                for sub in shape.shapes:
                    extract_from_shape(sub)
                return

            # 1) Standard text frame
            if hasattr(shape, "text_frame"):
                text = shape.text_frame.text.strip()
                if text:
                    content.append(text)
                    slide_has_content = True

            # 2) Simple text attribute
            elif hasattr(shape, "text"):
                text = shape.text.strip()
                if text:
                    content.append(text)
                    slide_has_content = True

            # 3) Native PPT tables
            if hasattr(shape, "has_table") and shape.has_table:
                content.append(f"\n[Table in Slide {slide_num}]")
                table = shape.table
                for row in table.rows:
                    cells = [cell.text.strip() for cell in row.cells]
                    if any(cells):
                        content.append(" | ".join(cells))
                slide_has_content = True

            # 4) Embedded Excel / OLE objects: get the CORRECT workbook for THIS shape
            if include_embedded_excels:
                excel_path = get_embedded_excel_path(shape, slide)
                if excel_path and excel_path in embedded_cache:
                    sheet_dfs = embedded_cache[excel_path]
                    wb_name = os.path.basename(excel_path)
                    
                    content.append(f"\n[Embedded workbook on Slide {slide_num}: {wb_name}]")
                    for sheet_name, df in sheet_dfs.items():
                        content.append(f"\n--- EMBEDDED SHEET: {sheet_name} ---")
                        content.append(df.head(max_rows).to_csv(index=False).strip())
                    slide_has_content = True

        # Run the helper for all top-level shapes
        for shape in slide.shapes:
            extract_from_shape(shape)

        # --------- Step 3: SmartArt / diagram text (plain text) ---------
        raw_smartart_texts = extract_smartart_texts(slide)

        if raw_smartart_texts:
            content.append(f"\n[SMARTART content on this slide]")
            for t in raw_smartart_texts:
                content.append(t)
            slide_has_content = True

        # --------- Step 4: Slide notes ---------
        if slide.has_notes_slide:
            try:
                notes_text = slide.notes_slide.notes_text_frame.text.strip()
                if notes_text:
                    content.append(f"\n[Notes]: {notes_text}")
                    slide_has_content = True
            except Exception:
                pass

        if not slide_has_content:
            content.append("[No text content found on this slide]")

    result = "\n".join(content)

    if not result.strip() or result.count("[No text content found on this slide]") == len(prs.slides):
        return f"[PowerPoint file has {len(prs.slides)} slides but no extractable text was found]"

    return result


### Processing all documents in the Azure storage folder

This cell scans the configured Azure Data Lake directory and processes every supported file type, including Word, Excel, and PowerPoint documents.  
For each file, the corresponding extraction function is applied to read text, tables, or embedded content.  
All extracted information is combined into a single text output, which is saved back into the same storage location for later use in the pipeline.  
The goal of this step is to unify the content of multiple document formats into one structured text file that can be passed to the AI model in the following stages.


In [0]:
import os
import xml.etree.ElementTree as ET

# Define what file types we want to process
extensions = [".pdf", ".pptx", ".ppt", ".docx", ".doc", ".xlsx", ".xls", ".csv"]
folder_path = INPUT_DIR

# List all files in the folder
items = dbutils.fs.ls(folder_path)

# Filter to only get files with our extensions
files = [item.path for item in items if any(item.name.lower().endswith(ext) for ext in extensions)]

print(f"Found {len(files)} files:", [os.path.basename(f) for f in files])

all_content = ''

for i, file_path in enumerate(files):
    all_content += f"\n\nFile {i+1}: {os.path.basename(file_path)}\n{'='*60}\n"
                                                                                                
    if file_path.lower().endswith('.docx'):
        docx_content = read_docx_text(file_path)
        all_content += docx_content + "\n"
    
    elif file_path.lower().endswith(".xlsx"):
        sheets = read_xlsx_sheets(file_path)
        for sheet_name, df in sheets.items():
            all_content += f"\n--- SHEET: {sheet_name} ---\n"
            all_content += df.head().to_csv(index=False) + "\n"

    elif file_path.lower().endswith('.pptx'):
        # Now this already includes embedded Excels in the correct slide position
        pptx_content = read_pptx_text(file_path, include_embedded_excels=True, max_rows=30)
        all_content += pptx_content + "\n"

    elif file_path.lower().endswith('.ppt'):
        all_content += "[.ppt format not supported by current extractor]\n"

output_path = f"{folder_path}/all_content.txt"
dbutils.fs.put(output_path, all_content, overwrite=True)

print(f"Combined content written to: {output_path}")


Found 1 files: ['TG3 Project Plan + Establishment v1.2.pptx']
Wrote 23812 bytes.
Combined content written to: abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/all_content.txt


In [0]:
if True: # set to True to test
    output_path = f"{INPUT_DIR}/all_content.txt"
    data = dbutils.fs.head(output_path, 100000000)
    print(data)



File 1: TG3 Project Plan + Establishment v1.2.pptx

--- Slide 1 ---
TG3Project Plan + EstablishmentNordic Energy Data Quality Improvement – Phase 1
Author: NordicTech Solutions AB – S&BS Team
Status Report Date: 28 Nov 2025

--- Slide 2 ---
TG3 Project Plan + Establishment
Project Information:






Revision History: 








Appendices:
Appendix 1: Meter Data Overview
Appendix 2: Data Cleaning Rules
Nordic Energy Data Quality Improvement – Phase 1
Novision Criteria: 
<Mark Novision Criteria by changing to Checkmark Bullets>

[Table in Slide 2]
Project ID | PRJ-NE-DQP-001 / CAT4-E-0785
Project Portfolio Owner | Emma Lindström
Project Sponsor | Jonas Hallberg
Project Manager | Maria Svensson
+46 70 123 45 67
maria.svensson@nordictechsolutions.com
SAP code | Not applicable
Project Period | 1 Nov 2025 – 28 Nov 2025

[Table in Slide 2]
Revision | Description of Change | Resp. | Effective Date
1.0 | First version | T. Carbin | 2025-11-01 (project start / first approved plan)
1.1 | Dumm

In [0]:
prompt = f"""
You are a project management data analyst extracting statuses from project documents. I will provide text extracted from a project management document, and you will extract status information from it.

CRITICAL INSTRUCTIONS:
- Only extract information explicitly present in the provided document.
- Do not make assumptions, estimates, or guesses.
- Do not invent or hallucinate values.
- If information is missing or unclear, do not generate a KPI for it.
- Keep wording and numbers as close as possible to the original source.

Pay attention to Status Report Date. Analyze the project and return:

1. A financial status score between 0 and 1, and a concise explanation of why
2. A timing status score between 0 and 1, and a concise explanation that explicitly uses the Status Report Date, compares the planned schedule (as described in the implementation plan in the document) with the actual progress reported, and clearly states whether the project is on track, slightly behind, significantly behind, or ahead of schedule, including a short justification for the chosen score.
3. The resource status must evaluate whether the project has the required roles, people, and support available as described in the document. The explanation should compare planned resource availability versus actual availability or constraints mentioned in the text, and state whether resources are sufficient, partially insufficient, or blocking the progress and mention based on which risks the value estimated. The score (0–1) should reflect this assessment.
4. A task status score between 0 and 1, and a concise explanation of why
5. An overall status score between 0 and 1, and a concise explanation of why

Respond only in valid JSON using this structure (no extra text, comments, or formatting outside the JSON):

{{
  "Financial": {{
    "StatusDescription": "<short explanation of financial status>",
    "Status": <number between 0 and 1>
  }},
  "Timing": {{
    "StatusDescription": "<short explanation of timing status>",
    "Status": <number between 0 and 1>
  }},
  "Resources": {{
    "StatusDescription": "<short explanation of resource status>",
    "Status": <number between 0 and 1>
  }},

    "Tasks": {{
    "Tasks Description In Project": "<short explanation of overall status with bulletpoints>",
    "Status": <number between 0 and 1>
  }},

  "Overall": {{
    "StatusDescription": "<short explanation of overall status>",
    "Status": <number between 0 and 1>
  }}
}}

Here is the project management document’s extracted text:
{all_content}
"""


In [0]:
DEPLOYMENT = "gpt-4o-sweden"
# DEPLOYMENT = "gpt-5-chat-latest"
response = client.chat.completions.create(
    model=DEPLOYMENT,
    messages=[
        {"role": "system", "content": "You are a project management analyst."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

In [0]:
models = client.models.list()

for m in models.data:
    print(m.id)

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-5.1-codex-max
gpt-5.1-2025-11-13
gpt-5.1
gpt-5.1-codex
gpt-5.1-codex-mini
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcri

In [0]:
import json
import re

raw_output = response.choices[0].message.content.strip()
# if the model returens markdown code blocks, remove them
if raw_output.startswith("```"):
    raw_output = re.sub(r'^```json?\s*', '', raw_output)
    raw_output = re.sub(r'```\s*$', '', raw_output)    

output = json.loads(raw_output)
output_path = f"{INPUT_DIR}/output.json"
output_json = json.dumps(output, indent=4, ensure_ascii=False)
dbutils.fs.put(output_path, output_json, overwrite=True)
print(f"Output JSON saved to: {output_path}")

Wrote 1612 bytes.
Output JSON saved to: abfss://itera-extraction-project@iterastorerm.dfs.core.windows.net/output.json


In [0]:
import json

json_str = dbutils.fs.head(f"{INPUT_DIR}/output.json", 100000)
raw = json.loads(json_str)

display(raw)


{'Financial': {'StatusDescription': 'Actual cost 115,000\u202fSEK vs planned 75,000\u202fSEK (≈40,000\u202fSEK over). Internal labor only, no external costs. Budget exceeded but ROI remains positive for pilot scope.',
  'Status': 0.4},
 'Timing': {'StatusDescription': 'Status Report Date 28\u202fNov\u202f2025 matches planned project end (1–28\u202fNov\u202f2025). Implementation plan shows Week\u202f1 Data Extraction completed, Weeks\u202f2–3 Data Cleaning in progress, Week\u202f4 Summary Report not started. At report date, final deliverables should be complete but are still pending, indicating the project is slightly behind schedule.',
  'Status': 0.7},
 'Resources': {'StatusDescription': 'Planned roles (Project Manager, Data Analyst, IT Technician, Energy Team) are assigned. Risks mention limited availability of Data Analyst and IT Technician, but overall resource exposure is low. Resources are partially insufficient due to occasional availability constraints.',
  'Status': 0.8},
 'Ta

In [0]:
from pyspark.sql import Row
from pyspark.sql import functions as F

rows = [
    Row(
        kpi_name="Financial",
        status_description=raw["Financial"]["StatusDescription"],
        status=float(raw["Financial"]["Status"])
    ),
    Row(
        kpi_name="Timing",
        status_description=raw["Timing"]["StatusDescription"],
        status=float(raw["Timing"]["Status"])
    ),
    Row(
        kpi_name="Resources",
        status_description=raw["Resources"]["StatusDescription"],
        status=float(raw["Resources"]["Status"])
    ),
    Row(
        kpi_name="Overall",
        status_description=raw["Overall"]["StatusDescription"],
        status=float(raw["Overall"]["Status"])
    ),
    Row(
        kpi_name="Tasks",
        status_description=raw["Tasks"]["Tasks Description In Project"],
        status=float(raw["Tasks"]["Status"])
    )
]

df = spark.createDataFrame(rows)

df = df.withColumn("ingested_at", F.current_timestamp())

# Write to a new Delta table
df.write.mode("overwrite").saveAsTable("project_status_kpi_long")


In [0]:
%sql
SELECT * 
FROM project_status_kpi_long
ORDER BY ingested_at DESC;


kpi_name,status_description,status,ingested_at
Tasks,• Data Extraction – Completed • Data Cleaning & Standardization – In Progress • Summary Report Preparation – Not Started (as of 28 Nov 2025) • Handover – Planned end of Week 4,0.6,2025-12-05T15:02:30.63Z
Resources,"Planned roles (Project Manager, Data Analyst, IT Technician, Energy Team) are assigned. Risks mention limited availability of Data Analyst and IT Technician, but overall resource exposure is low. Resources are partially insufficient due to occasional availability constraints.",0.8,2025-12-05T15:02:30.63Z
Timing,"Status Report Date 28 Nov 2025 matches planned project end (1–28 Nov 2025). Implementation plan shows Week 1 Data Extraction completed, Weeks 2–3 Data Cleaning in progress, Week 4 Summary Report not started. At report date, final deliverables should be complete but are still pending, indicating the project is slightly behind schedule.",0.7,2025-12-05T15:02:30.63Z
Overall,Project largely on track but slightly behind schedule and over budget. Core resources available though with minor constraints. Deliverables nearing completion but final reporting pending at status date.,0.6,2025-12-05T15:02:30.63Z
Financial,"Actual cost 115,000 SEK vs planned 75,000 SEK (≈40,000 SEK over). Internal labor only, no external costs. Budget exceeded but ROI remains positive for pilot scope.",0.4,2025-12-05T15:02:30.63Z


In [0]:
%sql
SHOW TABLES;
SELECT * FROM project_status_kpi_long;


kpi_name,status_description,status,ingested_at
Timing,"Status Report Date 28 Nov 2025 matches planned project end (1–28 Nov 2025). Implementation plan shows Week 1 Data Extraction completed, Weeks 2–3 Data Cleaning in progress, Week 4 Summary Report not started. At report date, final deliverables should be complete but are still pending, indicating the project is slightly behind schedule.",0.7,2025-12-05T15:02:30.63Z
Resources,"Planned roles (Project Manager, Data Analyst, IT Technician, Energy Team) are assigned. Risks mention limited availability of Data Analyst and IT Technician, but overall resource exposure is low. Resources are partially insufficient due to occasional availability constraints.",0.8,2025-12-05T15:02:30.63Z
Overall,Project largely on track but slightly behind schedule and over budget. Core resources available though with minor constraints. Deliverables nearing completion but final reporting pending at status date.,0.6,2025-12-05T15:02:30.63Z
Tasks,• Data Extraction – Completed • Data Cleaning & Standardization – In Progress • Summary Report Preparation – Not Started (as of 28 Nov 2025) • Handover – Planned end of Week 4,0.6,2025-12-05T15:02:30.63Z
Financial,"Actual cost 115,000 SEK vs planned 75,000 SEK (≈40,000 SEK over). Internal labor only, no external costs. Budget exceeded but ROI remains positive for pilot scope.",0.4,2025-12-05T15:02:30.63Z
